In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import numpy as np
from torchvision import models

/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/11507.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/3282.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/11014.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/8142.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/3614.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/5313.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/8106.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/8118.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/4104.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/8616.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/9740.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/1024.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/6280.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/5283.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/4864.mp3
/kaggle/input/compettioon-mp3/train_mp3s/train_mp3s/4920.mp3
/kaggle/input/competti

In [2]:
# process the label data
import numpy as np

# load training labels
label_path = '/kaggle/input/compettioon/train_label.npz'
label = np.load(label_path)

print(list(label.keys())) # labels
train_labels = label['labels']


print(type(train_labels)) #nd array
print(train_labels.shape) #(11886,)

['labels']
<class 'numpy.ndarray'>
(11886,)


In [3]:
# process the feature data
import numpy as np
# load feature
feature_path = '/kaggle/input/compettioon/feature_npz/feature_npz/'

feature_list = []

for i in range(11886):
    feature = np.load(feature_path + str(i) + "feature.npz")
#     print(list(feature.keys()))
    f = feature['arr_0'].reshape((299,40)) # 299*40
#     print(f.shape)
    feature_list.append(f)
    

feature_combined = np.stack(feature_list, axis=0)
print(feature_combined.shape) # (11886, 299, 40)

(11886, 299, 40)


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        # 确保 feature 是一个二维数组 [height, width]
        feature_tensor = torch.tensor(feature, dtype=torch.float32).unsqueeze(0)  # 添加通道维度，使其成为 [1, height, width]
        label_tensor = torch.tensor(label, dtype=torch.long)
        return feature_tensor, label_tensor




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_combined, train_labels, test_size=0.3, random_state=8)


dataset_train = MyDataset(X_train, y_train)
dataset_test = MyDataset(X_test, y_test)


# 使用 DataLoader 进行批处理和数据加载
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [5]:
# process the prediction data
import numpy as np
# load feature
feature_path_pred = '/kaggle/input/compettioon/feature_npz_test/feature_npz_test/'

feature_list_pred = []

for i in range(2447):
    feature_pred = np.load(feature_path_pred + str(i) + "feature.npz")
#     print(list(feature.keys()))
    f1 = feature_pred['arr_0'].reshape((299,40)) # 299*40
#     print(f1.shape)
    feature_list_pred.append(f1)
    

feature_combined_pred = np.stack(feature_list_pred, axis=0)
print(feature_combined_pred.shape) # (2447, 299, 40)
dataset_pred = MyDataset(feature_combined_pred,np.empty((2447, 0)))

pred_loader = torch.utils.data.DataLoader(dataset_pred, batch_size=batch_size, shuffle=False)

(2447, 299, 40)


In [6]:
def set_model(input_channels=1, num_classes=4):
    model = models.resnet34(pretrained=False)
    # 修改第一层卷积以适应输入数据的通道数
    model.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    # 修改全连接层输出为四个类别
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

model = set_model(input_channels=1, num_classes=4)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
optimizer = optim.AdamW(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_model(num_epochs, train_loader, test_loader):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')
        evaluate_model(test_loader)

def evaluate_model(test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on test set: {100 * correct / total}%')

train_model(num_epochs=26, train_loader=train_loader, test_loader=test_loader)


Epoch 1, Loss: 0.9123379805913339
Accuracy on test set: 71.87324733595065%
Epoch 2, Loss: 0.656989518724955
Accuracy on test set: 67.41446999439148%
Epoch 3, Loss: 0.577745706989215
Accuracy on test set: 74.81772293886708%
Epoch 4, Loss: 0.5267956749751017
Accuracy on test set: 72.32192933258553%
Epoch 5, Loss: 0.47624205843760414
Accuracy on test set: 76.38810992708918%
Epoch 6, Loss: 0.4439235137059138
Accuracy on test set: 75.79921480650589%
Epoch 7, Loss: 0.4148001302893345
Accuracy on test set: 79.92148065058889%
Epoch 8, Loss: 0.36882972224400595
Accuracy on test set: 78.79977565900168%
Epoch 9, Loss: 0.33835582458055935
Accuracy on test set: 77.20134604598991%
Epoch 10, Loss: 0.2996614911235296
Accuracy on test set: 80.4823331463825%
Epoch 11, Loss: 0.2852617377272019
Accuracy on test set: 77.7902411665732%
Epoch 12, Loss: 0.24660937281755302
Accuracy on test set: 81.15535614133483%
Epoch 13, Loss: 0.24330671549989627
Accuracy on test set: 78.40717891194616%
Epoch 14, Loss: 0.20

In [9]:
import pandas as pd
predictions = []
def predict_output(model,predict_loader,predictions):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)  # to GPU or CPU
    model.eval()
    with torch.no_grad():
        for inputs, labels in pred_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
#             print(predicted)

            if predicted.requires_grad:
                predicted = predicted.detach() # detach gradient computing
            if predicted.is_cuda:
                predicted = predicted.cpu()
                
            predicted = predicted.numpy()
#             print(predicted)
            predictions.append(predicted)
#             print(len(predicted))
#             print(len(predictions))
            concatenated_pred = np.concatenate(predictions, axis=0)
            
    return concatenated_pred
            
concatenated_pred = predict_output(model,pred_loader,predictions)

# print(concatenated_pred)
submission_df = pd.DataFrame({
    "id": range(len(concatenated_pred)),  # 创建一个从 0 到 预测数-1 的序列
    "category": concatenated_pred  # 使用预测结果填充类别列
})

# 保存 DataFrame 到 CSV 文件
submission_file_path = '/kaggle/working/submissionnnn.csv'  # 指定文件保存路径
submission_df.to_csv(submission_file_path, index=False)  # 确保不保存索引列

print(f"CSV file saved to {submission_file_path}")

CSV file saved to /kaggle/working/submissionnnn.csv
